In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import VGG16
from PIL import Image
import os
import pandas as pd
from sklearn.model_selection import train_test_split

# Konfigurasi
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 32
FEATURE_DIM = 512

# Path ke dataset
base_dir = r"D:\Semester7\CV\identifikasi_kain\data\raw"  # Ganti dengan path dataset Anda
nir_dir = os.path.join(base_dir, "NIR")
rgb_dir = os.path.join(base_dir, "RGB")

# Path untuk output
output_dir = r"D:\Semester7\CV\identifikasi_kain\outputs\cnn_VGG16_extracted_features"  # Ganti dengan path output yang diinginkan
os.makedirs(output_dir, exist_ok=True)

# Daftar kelas
classes = ["katun", "polyester", "sutra", "rayon"]

# Fungsi untuk memuat dan memproses gambar
def load_and_preprocess_image(image_path, is_nir=False):
    # Baca gambar
    img = Image.open(image_path)
    
    # Konversi ke array numpy
    img_array = np.array(img)
    
    # Jika gambar NIR (grayscale), ubah ke 3 channel
    if is_nir:
        if len(img_array.shape) == 2:
            img_array = np.stack((img_array,) * 3, axis=-1)
        elif img_array.shape[2] == 1:
            img_array = np.repeat(img_array, 3, axis=2)
    
    # Resize gambar
    img_array = tf.image.resize(img_array, [IMG_HEIGHT, IMG_WIDTH]).numpy()
    
    # Normalisasi nilai pixel
    img_array = img_array / 255.0
    
    return img_array

# Fungsi untuk memuat semua gambar dari folder
def load_images_from_folder(base_folder, is_nir=False):
    images = []
    labels = []
    filenames = []
    
    for class_idx, class_name in enumerate(classes):
        class_folder = os.path.join(base_folder, class_name)
        
        if not os.path.exists(class_folder):
            print(f"Folder {class_folder} tidak ditemukan. Melanjutkan...")
            continue
            
        for filename in os.listdir(class_folder):
            if filename.endswith(('.jpg', '.jpeg', '.png', '.JPG')):
                img_path = os.path.join(class_folder, filename)
                try:
                    img = load_and_preprocess_image(img_path, is_nir)
                    images.append(img)
                    labels.append(class_idx)
                    filenames.append(filename)
                except Exception as e:
                    print(f"Error memproses {img_path}: {str(e)}")
    
    return np.array(images), np.array(labels), filenames

# Memuat gambar NIR dan RGB
print("Memuat gambar NIR...")
nir_images, nir_labels, nir_filenames = load_images_from_folder(nir_dir, is_nir=True)

print("Memuat gambar RGB...")
rgb_images, rgb_labels, rgb_filenames = load_images_from_folder(rgb_dir, is_nir=False)

print(f"Jumlah gambar NIR: {len(nir_images)}")
print(f"Jumlah gambar RGB: {len(rgb_images)}")

# Membuat model untuk ekstraksi fitur
def create_feature_extractor():
    # Gunakan VGG16 pre-trained sebagai feature extractor
    base_model = VGG16(weights='imagenet', include_top=False, 
                       input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))
    
    # Tambahkan global average pooling
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    
    # Buat model
    model = keras.Model(inputs=base_model.input, outputs=x)
    
    return model

# Membuat feature extractor
feature_extractor = create_feature_extractor()

# Mengekstraksi fitur dari gambar NIR
print("Mengekstraksi fitur dari gambar NIR...")
nir_features = feature_extractor.predict(nir_images, verbose=1)

# Mengekstraksi fitur dari gambar RGB
print("Mengekstraksi fitur dari gambar RGB...")
rgb_features = feature_extractor.predict(rgb_images, verbose=1)

# Menyimpan fitur yang diekstraksi
def save_features(features, labels, filenames, output_file):
    # Buat DataFrame untuk menyimpan fitur
    feature_df = pd.DataFrame(features)
    feature_df['label'] = labels
    feature_df['filename'] = filenames
    
    # Simpan ke file CSV
    feature_df.to_csv(output_file, index=False)
    print(f"Fitur disimpan di: {output_file}")

# Menyimpan fitur NIR dan RGB ke direktori yang ditentukan
nir_output_path = os.path.join(output_dir, "nir_features.csv")
rgb_output_path = os.path.join(output_dir, "rgb_features.csv")

save_features(nir_features, nir_labels, nir_filenames, nir_output_path)
save_features(rgb_features, rgb_labels, rgb_filenames, rgb_output_path)

print("Ekstraksi fitur selesai!")

# # Optional: Membuat model klasifikasi dari fitur yang diekstraksi
# def create_classifier(input_dim, num_classes):
#     model = keras.Sequential([
#         layers.Dense(128, activation='relu', input_shape=(input_dim,)),
#         layers.Dropout(0.5),
#         layers.Dense(64, activation='relu'),
#         layers.Dropout(0.3),
#         layers.Dense(num_classes, activation='softmax')
#     ])
    
#     model.compile(optimizer='adam',
#                   loss='sparse_categorical_crossentropy',
#                   metrics=['accuracy'])
    
#     return model

# # Contoh penggunaan fitur untuk klasifikasi (menggunakan fitur NIR)
# if len(nir_features) > 0:
#     X_train, X_test, y_train, y_test = train_test_split(
#         nir_features, nir_labels, test_size=0.2, random_state=42
#     )
    
#     nir_classifier = create_classifier(nir_features.shape[1], len(classes))
#     nir_classifier.fit(X_train, y_train, 
#                        epochs=10, 
#                        validation_data=(X_test, y_test),
#                        verbose=1)
    
#     # Evaluasi model
#     test_loss, test_acc = nir_classifier.evaluate(X_test, y_test, verbose=0)
#     print(f"Akurasi model NIR: {test_acc:.4f}")

# # Hal serupa dapat dilakukan untuk fitur RGB
# if len(rgb_features) > 0:
#     X_train, X_test, y_train, y_test = train_test_split(
#         rgb_features, rgb_labels, test_size=0.2, random_state=42
#     )
    
#     rgb_classifier = create_classifier(rgb_features.shape[1], len(classes))
#     rgb_classifier.fit(X_train, y_train, 
#                        epochs=10, 
#                        validation_data=(X_test, y_test),
#                        verbose=1)
    
#     # Evaluasi model
#     test_loss, test_acc = rgb_classifier.evaluate(X_test, y_test, verbose=0)
#     print(f"Akurasi model RGB: {test_acc:.4f}")

Memuat gambar NIR...
Memuat gambar RGB...
Jumlah gambar NIR: 120
Jumlah gambar RGB: 120
Mengekstraksi fitur dari gambar NIR...
4/4 ━━━━━━━━━━━━━━━━━━━━ 19s 5s/step
Mengekstraksi fitur dari gambar RGB...
4/4 ━━━━━━━━━━━━━━━━━━━━ 19s 5s/step
Fitur disimpan di: D:\Semester7\CV\identifikasi_kain\outputs\cnn_VGG16_extracted_features\nir_features.csv
Fitur disimpan di: D:\Semester7\CV\identifikasi_kain\outputs\cnn_VGG16_extracted_features\rgb_features.csv
Ekstraksi fitur selesai!


In [3]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import VGG16
from PIL import Image
import os
import pandas as pd
from sklearn.model_selection import train_test_split

# Konfigurasi
IMG_HEIGHT = 224
IMG_WIDTH = 224

# Path ke dataset
base_dir = r"D:\Semester7\CV\identifikasi_kain\data\raw"  # Ganti dengan path dataset Anda
nir_dir = os.path.join(base_dir, "NIR")
rgb_dir = os.path.join(base_dir, "RGB")

# Path untuk output
output_dir = r"D:\Semester7\CV\identifikasi_kain\outputs\cnn_VGG16_extracted_features"  # Ganti dengan path output yang diinginkan
os.makedirs(output_dir, exist_ok=True)

# Daftar kelas
classes = ["katun", "polyester", "sutra", "rayon"]

# Fungsi untuk menghitung nilai rata-rata RGB
def calculate_rgb_average(image_path):
    # Baca gambar
    img = Image.open(image_path)
    
    # Konversi ke array numpy
    img_array = np.array(img)
    
    # Jika gambar grayscale, konversi ke RGB
    if len(img_array.shape) == 2:
        img_array = np.stack((img_array,) * 3, axis=-1)
    elif img_array.shape[2] == 1:
        img_array = np.repeat(img_array, 3, axis=2)
    
    # Hitung nilai rata-rata untuk setiap channel
    r_avg = np.mean(img_array[:, :, 0])
    g_avg = np.mean(img_array[:, :, 1])
    b_avg = np.mean(img_array[:, :, 2])
    
    return r_avg, g_avg, b_avg

# Fungsi untuk menghitung nilai rata-rata NIR
def calculate_nir_average(image_path):
    # Baca gambar
    img = Image.open(image_path)
    
    # Konversi ke array numpy
    img_array = np.array(img)
    
    # Jika gambar memiliki lebih dari satu channel, ambil channel pertama sebagai NIR red sebagai proxy
    if len(img_array.shape) > 2:
        nir_avg = np.mean(img_array[:, :, 0])  # Ambil channel pertama
    else:
        nir_avg = np.mean(img_array)  # Jika grayscale
    
    return nir_avg

# Fungsi untuk memuat semua gambar dari folder dan menghitung nilai rata-ratanya
def process_images_from_folder(base_folder, is_nir=False):
    data = []
    
    for class_idx, class_name in enumerate(classes):
        class_folder = os.path.join(base_folder, class_name)
        
        if not os.path.exists(class_folder):
            print(f"Folder {class_folder} tidak ditemukan. Melanjutkan...")
            continue
            
        for filename in os.listdir(class_folder):
            if filename.endswith(('.jpg', '.jpeg', '.png', '.JPG')):
                img_path = os.path.join(class_folder, filename)
                try:
                    if is_nir:
                        nir_avg = calculate_nir_average(img_path)
                        data.append({
                            'filename': filename,
                            'label': class_name,
                            'nir_avg': nir_avg
                        })
                    else:
                        r_avg, g_avg, b_avg = calculate_rgb_average(img_path)
                        data.append({
                            'filename': filename,
                            'label': class_name,
                            'r_avg': r_avg,
                            'g_avg': g_avg,
                            'b_avg': b_avg
                        })
                except Exception as e:
                    print(f"Error memproses {img_path}: {str(e)}")
    
    return data

# Memproses gambar NIR dan RGB
print("Memproses gambar NIR...")
nir_data = process_images_from_folder(nir_dir, is_nir=True)

print("Memproses gambar RGB...")
rgb_data = process_images_from_folder(rgb_dir, is_nir=False)

print(f"Jumlah gambar NIR: {len(nir_data)}")
print(f"Jumlah gambar RGB: {len(rgb_data)}")

# Menyimpan data ke CSV
def save_to_csv(data, output_file, is_nir=False):
    # Buat DataFrame
    df = pd.DataFrame(data)
    
    # Simpan ke file CSV
    df.to_csv(output_file, index=False)
    print(f"Data disimpan di: {output_file}")
    
    # Tampilkan preview data
    print(f"Preview data {os.path.basename(output_file)}:")
    print(df.head())

# Menyimpan data NIR dan RGB ke direktori yang ditentukan
nir_output_path = os.path.join(output_dir, "nir_averages.csv")
rgb_output_path = os.path.join(output_dir, "rgb_averages.csv")

save_to_csv(nir_data, nir_output_path, is_nir=True)
save_to_csv(rgb_data, rgb_output_path, is_nir=False)

print("Ekstraksi nilai rata-rata selesai!")

Memproses gambar NIR...
Memproses gambar RGB...
Jumlah gambar NIR: 120
Jumlah gambar RGB: 120
Data disimpan di: D:\Semester7\CV\identifikasi_kain\outputs\cnn_VGG16_extracted_features\nir_averages.csv
Preview data nir_averages.csv:
           filename  label     nir_avg
0  katun_nir_01.jpg  katun  125.524595
1  katun_nir_02.jpg  katun  125.161869
2  katun_nir_03.jpg  katun  125.266383
3  katun_nir_04.jpg  katun  125.329034
4  katun_nir_05.jpg  katun  124.979867
Data disimpan di: D:\Semester7\CV\identifikasi_kain\outputs\cnn_VGG16_extracted_features\rgb_averages.csv
Preview data rgb_averages.csv:
           filename  label       r_avg       g_avg       b_avg
0  katun_rgb_01.jpg  katun  170.406291  166.408992  163.289820
1  katun_rgb_02.jpg  katun  169.900813  165.945546  162.863991
2  katun_rgb_03.jpg  katun  169.696149  165.714474  162.687128
3  katun_rgb_04.jpg  katun  169.227814  165.259479  162.397206
4  katun_rgb_05.jpg  katun  170.427607  166.687460  163.789242
Ekstraksi nilai rata